In [1]:
import pandas as pd
from pandas import datetime
import io as io
from io import StringIO

In [2]:
cd timeSeriesDataset/

C:\Users\Wan Fikri\Anaconda Script\COVID-19\LSTM_COVID\timeSeriesDataset


In [3]:
confirmed_df = pd.read_csv("confirmed_df.csv")
deaths_df = pd.read_csv("deaths_df.csv")
recoveries_df = pd.read_csv("recoveries_df.csv")

In [4]:
confirmed_df

,Province/State,Country/Region,Lat,Long,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,...,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20
0,NaN,Afghanistan,33.939110,67.709953,996,1026,1092,1176,1279,1351,...,28424,28833,29157,29481,29640,30175,30451,30616,30967,31238
1,NaN,Albania,41.153300,20.168300,562,584,609,634,663,678,...,1891,1962,1995,2047,2114,2192,2269,2330,2402,2466
2,NaN,Algeria,28.033900,1.659600,2629,2718,2811,2910,3007,3127,...,11631,11771,11920,12076,12248,12445,12685,12968,13273,13571
3,NaN,Andorra,42.506300,1.521800,713,717,717,723,723,731,...,855,855,855,855,855,855,855,855,855,855
4,NaN,Angola,-11.202700,17.873900,24,24,24,25,25,25,...,176,183,186,189,197,212,212,259,267,276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,NaN,Sao Tome and Principe,0.186400,6.613100,4,4,4,4,4,4,...,698,698,702,707,710,711,712,713,713,713
262,NaN,Yemen,15.552727,48.516388,1,1,1,1,1,1,...,922,941,967,992,1015,1076,1089,1103,1118,1128
263,NaN,Comoros,-11.645500,43.333300,0,0,0,0,0,0,...,247,247,247,265,265,272,272,272,272,272
264,NaN,Tajikistan,38.861000,71.276100,0,0,0,0,0,0,...,5399,5457,5513,5567,5630,5691,5747,5799,5849,5900


In [5]:
# Drop date columns if they are mostly NaN
na_columns = (confirmed_df.isna().sum() / confirmed_df.shape[0]) > 0.99
na_columns = na_columns[na_columns]

confirmed_df = confirmed_df.drop(na_columns.index, axis=1)
deaths_df = deaths_df.drop(na_columns.index, axis=1)
recoveries_df = recoveries_df.drop(na_columns.index, axis=1)

## rs=['Country/Region', 'ProvinceTidy up the data
confirmed_df = confirmed_df.melt(id_vars=['Country/Region', 'Province/State', 'Lat', 'Long'], var_name='date', value_name='confirmed')
deaths_df = deaths_df.melt(id_vars =['Country/Region', 'Province/State', 'Lat', 'Long'], var_name='date', value_name='deaths')
recoveries_df = recoveries_df.melt(id_vars=['Country/Region', 'Province/State', 'Lat', 'Long'], var_name='date', value_name='recoveries')
                           
confirmed_df['date'] = pd.to_datetime(confirmed_df['date'])
deaths_df['date'] = pd.to_datetime(deaths_df['date'])
recoveries_df['date'] = pd.to_datetime(recoveries_df['date'])
full_df = confirmed_df.merge(recoveries_df).merge(deaths_df)
full_df['Id'] = full_df.index+1
full_df = full_df.rename(columns={'Country/Region': 'Country_Region', 'date': 'Date', 'confirmed': 'ConfirmedCases', 'recoveries': 'Recoveries', 'deaths': 'Fatalities', 'Province/State': 'Province_State'})
full_df.drop(full_df.columns[[2, 3]], axis = 1, inplace = True) 
# Check null values
full_df.isnull().sum()

Country_Region        0
Province_State    13104
Date                  0
ConfirmedCases        0
Recoveries            0
Fatalities            0
Id                    0
dtype: int64

In [6]:
full_df

,Country_Region,Province_State,Date,ConfirmedCases,Recoveries,Fatalities,Id
0,Afghanistan,NaN,2020-04-19,996,131,33,1
1,Albania,NaN,2020-04-19,562,314,26,2
2,Algeria,NaN,2020-04-19,2629,1047,375,3
3,Andorra,NaN,2020-04-19,713,235,36,4
4,Angola,NaN,2020-04-19,24,6,2,5
...,...,...,...,...,...,...,...
17779,Sao Tome and Principe,NaN,2020-06-29,713,235,13,17780
17780,Yemen,NaN,2020-06-29,1128,432,304,17781
17781,Comoros,NaN,2020-06-29,272,161,7,17782
17782,Tajikistan,NaN,2020-06-29,5900,4506,52,17783


In [7]:
testing_df = full_df.copy()
testing_df = testing_df.rename(columns={'Id': 'ForecastId'})
testing_df.drop(testing_df.columns[[3, 4, 5]], axis = 1, inplace = True) 

In [8]:
testing_df

,Country_Region,Province_State,Date,ForecastId
0,Afghanistan,NaN,2020-04-19,1
1,Albania,NaN,2020-04-19,2
2,Algeria,NaN,2020-04-19,3
3,Andorra,NaN,2020-04-19,4
4,Angola,NaN,2020-04-19,5
...,...,...,...,...
17779,Sao Tome and Principe,NaN,2020-06-29,17780
17780,Yemen,NaN,2020-06-29,17781
17781,Comoros,NaN,2020-06-29,17782
17782,Tajikistan,NaN,2020-06-29,17783


In [9]:
testing_df.to_csv('testing_LSTM(1).csv', index = False, header = True)